In [1]:
# So I am thinking we take what we did before, where we applied the neural network we did that simply does multiple stuff, and we do it simply, but I am thinking what if we take that, and instead we actually apply to something like language.
# So for example, we use the model, and we train on a bunch of characters to basically predict the next. 
# I guess the idea is we take the inputs, which are just characters, we make them into 30D vector, 26 of them is just for the letter
# So for example
# A is [1, 0, 0, 0, etc 26 times] + [0, 0, 1, etc..] and we concatinate it so its [1, 0, 0... 26 and then 0, 0, 1, etc.. for the position in the sentence of where it is] (this is depending on the sliding window say we have a character window for
# our model of 250 characters)
# Ok, then we simply put this into a huge input thing vectorized, and we concatinate with all the other letters. And now we train a model on this thing. And it will be having 26 outputs. And each is the letter it should output.
# Ok. Cool. So what next? Well, now this should be it. We train on the entire text and thats it. This is a baby kind of thing as an intro to language models. I know these are different but whatever sounds cool!

# So what do u think of this?

In [2]:
import re

def extract_letters_and_spaces(input_string):
    # Use regular expression to keep only letters and spaces
    result = re.sub(r'[^a-zA-Z\s]', '', input_string)
    return result

In [3]:
# FIRST ATTEMPT
import numpy as np
# Ok so we need a way to encode this. Lets do that. Then we put that into our model, and train on that, with each thing outputting each letter thing. And then just go from there.

#corpus = 
#cleaned_string = extract_letters_and_spaces(corpus).upper()
#print(cleaned_string)


In [4]:
context_window = 250
#cleaned_string[:context_window]

In [5]:
# Wait so for our training thing we are gonna need way more right? Like this this thing might not learn... i guess you gonna have different training samples with this context window, and feed it multiple times with all your training. This means you might need to cut up the text of catcher in the rye for example, into sequences like this, and just u know, train the model as you go through

In [6]:
# So the first thing we need is the data, which is the entire corpus chopped up into 250 blocks, and predictions like "next letter" type thing. 
with open("the_catcher_in_the_rye.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [7]:
big_corpus = extract_letters_and_spaces(text).replace("\n", " ").upper()

In [8]:
cleaned_corpus = []
for c in big_corpus:
    if c in ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", " "]:
        cleaned_corpus.append(c)


In [9]:
# ok so now we need to do our encoding.
import numpy as np
def encoder(character, context_window, where_is_character_in_context):
    encoder_book = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", " "]
    encoded_c = []
    for i in range(len(encoder_book)):
        if encoder_book[i]==character:
            encoded_c.append(1)
        else:
            encoded_c.append(0)
    # then append the position and thats based on the context window
    for p in range(context_window):
        if p==where_is_character_in_context:
            encoded_c.append(1)
        else:
            encoded_c.append(0)
    
    return encoded_c

# Example
encoder('E', 5, 1) # (e in hello) make sure you put the index. so i wanna encode entire words


[0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0]

In [10]:
# this should simply do the following: given the entir elist of the coprus, we should simply chop up the corpus so its 250 sort of blocks, we need to change our original nn cuz it took pixels kind of stuff. We are gonna already be in a vecotrized form.
# anyways this function simply give the list of corpus, chops up the corpus so its 250 blocks, then basically does the whole thing so we have training data and basically puts the y_labels with next "right" character. (and finally encodes it, then does the numpy stuff for the right format and boom)
def reformat_for_training(corp, context_window):
    current_data_sample = []
    # it should be a tuple thing of tuples here
    labels = []
    for i in range(context_window, len(corp)):
        current_window_data = corp[(i-context_window):i]
        current_data_sample.append(current_window_data)
        labels.append(corp[i])
    return current_data_sample, labels

        

In [11]:
string_data, labels = reformat_for_training(cleaned_corpus, 250)

In [12]:
#labels[0]

In [13]:
#string_data[:2]

In [14]:
# ok now that works let us encode the string data into actual numbers ang go from there
import tqdm
def encode_corpus_data(ll_of_strings, context_size):
    encoded_string_data = []
    for sample in tqdm.tqdm(ll_of_strings):
        current_sample = []
        for chari in range(len(sample)):
            current_sample.append(encoder(sample[chari], context_size, chari)) # note this will simply append it all, and this basically becomes our actual vecotrized input! it becomes not just for 1 character, but now 1 giant vector for all
        encoded_string_data.append(current_sample)
    return np.array(encoded_string_data)
X = encode_corpus_data(string_data, context_window)  

100%|█████████████████████████████████████████████████████████████████████████████| 1443/1443 [00:03<00:00, 388.42it/s]


In [15]:
X.shape

(1443, 250, 277)

In [26]:
X[0].shape

(250, 277)

In [17]:
# ok this is really good now. Now let us feed this into our neural network
def correct_labels(lab):
    output_book = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", " "]
    correct_label = []
    for l in lab:
        correct_label.append(output_book.index(l))
    return correct_label
        

# we also have our ylabels they are:
y = np.array(correct_labels(labels))
y

array([26,  5,  4, ..., 26, 12,  4])

In [18]:
# for labels we can simply output something like we can simply have instead k classes so we can just output a number between 0 and 26, each corresponding to the right index of the 


In [19]:
# OUR MODEL - Modified for multi-class output

LAYERS = [(69250, 1024),    # Drastic reduction
          (1024, 512),      # Mid-level features
          (512, 256),       # Compact representation
          (256, 27)]        # Output layer now has 27 units (one per class)

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import pickle
import numpy as np

class FastANN(nn.Module):
    def __init__(self, layer_sizes):
        super().__init__()
        layers = []
        for i, (in_size, out_size) in enumerate(layer_sizes):
            layers.append(nn.Linear(in_size, out_size))
            if i < len(layer_sizes)-1:
                layers.append(nn.ReLU())
            # No activation on final layer - we'll use CrossEntropyLoss which includes SoftMax
        self.net = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.net(x)

def train_model(X, y, epochs=2500, batch_size=1000, lr=0.01):
    device = torch.device("cuda")#("cpu")  # or "cuda" if available
    
    # Convert to PyTorch tensors
    X_tensor = torch.FloatTensor(X.reshape(X.shape[0], -1)).to(device)
    y_tensor = torch.LongTensor(y).to(device)  # Class indices as long integers
    
    # Create model with GPU support
    layer_sizes = LAYERS
    model = FastANN(layer_sizes).to(device)
    
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()  # Combines LogSoftMax and NLLLoss
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # Data loader
    dataset = TensorDataset(X_tensor, y_tensor)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # Training loop
    model.train()
    for epoch in tqdm(range(epochs), desc="Training model"):
        for batch_X, batch_y in loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
    
    # Save weights and biases
    params = {
        "weights": [layer.weight.data.cpu().numpy() 
                   for layer in model.net if isinstance(layer, nn.Linear)],
        "biases": [layer.bias.data.cpu().numpy() 
                  for layer in model.net if isinstance(layer, nn.Linear)]
    }
    
    with open('letter_model_params.pkl', 'wb') as f:
        pickle.dump(params, f)
    
    return model

# Load data
# X = your input data (n_samples, height, width)
# y = your labels as integers (0-26 corresponding to output_book)

output_book = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", 
               "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", " "]

# Train the single model
print("\n=== Training single model for all letters ===")
model = train_model(X, y)

# To get predictions:
# model.eval()
# with torch.no_grad():
#     outputs = model(input_tensor)
#     probabilities = torch.softmax(outputs, dim=1)  # Convert to probabilities
#     predicted_class = torch.argmax(probabilities, dim=1)


=== Training single model for all letters ===


Training model: 100%|██████████████████████████████████████████████████████████████| 2500/2500 [00:55<00:00, 45.14it/s]


In [28]:
import torch
import numpy as np
from torch import nn

class FastANN(nn.Module):
    def __init__(self, layer_sizes):
        super().__init__()
        layers = []
        for i, (in_size, out_size) in enumerate(layer_sizes):
            layers.append(nn.Linear(in_size, out_size))
            if i < len(layer_sizes)-1:
                layers.append(nn.ReLU())
        self.net = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.net(x)

def load_model(model_path='letter_model_params.pkl'):
    # Define the same architecture used during training
    layer_sizes = LAYERS
    model = FastANN(layer_sizes)
    
    # Load the saved parameters
    with open(model_path, 'rb') as f:
        params = pickle.load(f)
    
    # Set the model weights
    for i, layer in enumerate([layer for layer in model.net if isinstance(layer, nn.Linear)]):
        layer.weight.data = torch.FloatTensor(params['weights'][i])
        layer.bias.data = torch.FloatTensor(params['biases'][i])
    
    return model

def predict_letter(model, input_data, output_book):
    """
    Predicts the letter from input data
    
    Args:
        model: The trained FastANN model
        input_data: Input array (1, height, width) or flattened (69250,)
        output_book: List of possible outputs (A-Z + space)
    
    Returns:
        tuple: (predicted_letter, confidence, all_probabilities)
    """
    # Ensure model is in evaluation mode
    model.eval()
    
    # Convert input to tensor and flatten if needed
    if len(input_data.shape) > 1:
        input_data = input_data.reshape(1, -1)  # Flatten to (1, 69250)
    input_tensor = torch.FloatTensor(input_data)
    
    # Get predictions
    with torch.no_grad():
        outputs = model(input_tensor)
        probabilities = torch.softmax(outputs, dim=1)
    
    # Get top prediction
    top_prob, top_idx = torch.max(probabilities, dim=1)
    predicted_letter = output_book[top_idx.item()]
    confidence = top_prob.item()
    
    # Get all probabilities as a dictionary
    all_probs = {output_book[i]: prob.item() for i, prob in enumerate(probabilities.squeeze())}
    
    return predicted_letter, confidence, all_probs

In [77]:
# What do you want your model to predict? 
# Function that takes in any string input and basically converts it in the right format, and then we should have a loop, where our function outputs some text (say maybe 100 characters or something)
import tqdm
def encode_input(user_input):
    if len(user_input) < 250:
        user_input = " "*250 + user_input
    big_corpus = extract_letters_and_spaces(user_input).replace("\n", " ").upper()
    
    cleaned_corpus = []
    for c in big_corpus:
        if c in ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", " "]:
            cleaned_corpus.append(c)
    
    string_data, labels = reformat_for_training(cleaned_corpus, 250)
    #print(cleaned_corpus)
    return encode_corpus_data(string_data, context_window) # limit of 250 characters for now but lets see how it predicts         

In [78]:
user_input = "It was about this little kid that wouldn’t let anybody look at his goldfish and it was"
X = encode_input(user_input)

100%|█████████████████████████████████████████████████████████████████████████████████| 85/85 [00:00<00:00, 268.89it/s]


In [79]:
X[-1] # truthfully we only want the last one cuz thats the next 

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [1, 0, 0, ..., 0, 0, 1]])

In [83]:
# Load your model
model = load_model('letter_model_params.pkl')

# Your output book (must match training)
output_book = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", 
               "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", " "]

complete_sentence = "If you really want to hear about it, the first thing you’ll probably want to know is where I was born, an what my lousy childhood was like, and how my parents were occupied and all before they had me, and all that David Copperfield kind of crap, but I don’t feel like going into it, if you want to know the truth. In the first place, that stuff bores me, and in the second place, my parents would have about two hemorrhages apiece if I told anything pretty personal about them. They’re quite touchy about anything like that, especially my father. They’re nice and all — I’m not saying that — but they’re also touchy as hell. Besides, I’m not going to tell you my whole goddam autobiography or anything. I’ll just"
for nextten in range(30):

    X = encode_input(complete_sentence)
    
    # Example input data (replace with your actual data)
    # Should be either:
    # - (height, width) shaped array (will be flattened automatically)
    # - Already flattened array of length 69250
    example_input = X[-1] #np.random.rand(1, 69250)  # Replace with your actual data
    
    # Make prediction
    letter, confidence, all_probs = predict_letter(model, example_input, output_book)
    
    #print(f"Predicted letter: {letter} with confidence {confidence:.2%}")
    #print("\nAll probabilities:")
    
    for char, prob in sorted(all_probs.items(), key=lambda x: x[1], reverse=True)[:5]:
        complete_sentence = complete_sentence + char
        #print(f"{char}: {prob:.2%}")
        break
    print(complete_sentence)

100%|███████████████████████████████████████████████████████████████████████████████| 434/434 [00:01<00:00, 289.55it/s]


If you really want to hear about it, the first thing you’ll probably want to know is where I was born, an what my lousy childhood was like, and how my parents were occupied and all before they had me, and all that David Copperfield kind of crap, but I don’t feel like going into it, if you want to know the truth. In the first place, that stuff bores me, and in the second place, my parents would have about two hemorrhages apiece if I told anything pretty personal about them. They’re quite touchy about anything like that, especially my father. They’re nice and all — I’m not saying that — but they’re also touchy as hell. Besides, I’m not going to tell you my whole goddam autobiography or anything. I’ll justT


100%|███████████████████████████████████████████████████████████████████████████████| 435/435 [00:01<00:00, 293.53it/s]


If you really want to hear about it, the first thing you’ll probably want to know is where I was born, an what my lousy childhood was like, and how my parents were occupied and all before they had me, and all that David Copperfield kind of crap, but I don’t feel like going into it, if you want to know the truth. In the first place, that stuff bores me, and in the second place, my parents would have about two hemorrhages apiece if I told anything pretty personal about them. They’re quite touchy about anything like that, especially my father. They’re nice and all — I’m not saying that — but they’re also touchy as hell. Besides, I’m not going to tell you my whole goddam autobiography or anything. I’ll justT 


100%|███████████████████████████████████████████████████████████████████████████████| 436/436 [00:01<00:00, 288.93it/s]


...

If you really want to hear about it, the first thing you’ll probably want to know is where I was born, an what my lousy childhood was like, and how my parents were occupied and all before they had me, and all that David Copperfield kind of crap, but I don’t feel like going into it, if you want to know the truth. In the first place, that stuff bores me, and in the second place, my parents would have about two hemorrhages apiece if I told anything pretty personal about them. They’re quite touchy about anything like that, especially my father. They’re nice and all — I’m not saying that — but they’re also touchy as hell. Besides, I’m not going to tell you my whole goddam autobiography or anything. I’ll justT TELL YOL ABOUT THI  RADMAN


100%|███████████████████████████████████████████████████████████████████████████████| 462/462 [00:01<00:00, 289.50it/s]


If you really want to hear about it, the first thing you’ll probably want to know is where I was born, an what my lousy childhood was like, and how my parents were occupied and all before they had me, and all that David Copperfield kind of crap, but I don’t feel like going into it, if you want to know the truth. In the first place, that stuff bores me, and in the second place, my parents would have about two hemorrhages apiece if I told anything pretty personal about them. They’re quite touchy about anything like that, especially my father. They’re nice and all — I’m not saying that — but they’re also touchy as hell. Besides, I’m not going to tell you my whole goddam autobiography or anything. I’ll justT TELL YOL ABOUT THI  RADMAN 


100%|███████████████████████████████████████████████████████████████████████████████| 463/463 [00:01<00:00, 292.60it/s]


If you really want to hear about it, the first thing you’ll probably want to know is where I was born, an what my lousy childhood was like, and how my parents were occupied and all before they had me, and all that David Copperfield kind of crap, but I don’t feel like going into it, if you want to know the truth. In the first place, that stuff bores me, and in the second place, my parents would have about two hemorrhages apiece if I told anything pretty personal about them. They’re quite touchy about anything like that, especially my father. They’re nice and all — I’m not saying that — but they’re also touchy as hell. Besides, I’m not going to tell you my whole goddam autobiography or anything. I’ll justT TELL YOL ABOUT THI  RADMAN S
